In [1]:
# 400 meter coordinate accuracy
# 1923 earliest year
import pandas as pd

pd.set_option("display.max_columns", None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)


In [2]:
species_path = input("Enter the file path: ")

# species = pd.read_parquet(species_path)
species = pd.read_json(species_path)


In [ ]:
species


In [3]:
ecoregions_path = input("Enter the file path: ")

ecoregions = pd.read_json(ecoregions_path)


In [ ]:
ecoregions


In [4]:
# reset index of species and ecoregions
species.reset_index(drop=True, inplace=True)
ecoregions.reset_index(drop=True, inplace=True)


In [5]:
rankedArray = []
for i in range(len(species)):
    result = species.loc[i]
    rankedResult = {}
    for curr in result["unique_id"]:
        rankedResult[curr] = rankedResult.get(curr, 0) + 1
    rankedArray.append(rankedResult)

rankedObject = {}
for item in rankedArray:
    for key in item:
        if key not in rankedObject:
            rankedObject[key] = item[key]
        else:
            rankedObject[key] += item[key]

In [6]:
ranked_df = pd.DataFrame.from_dict(
    rankedObject, orient="index", columns=["species_count"]
)

ranked_df["unique_id"] = ranked_df.index

# drop row where unique_id = '497' LOOK INTO WHY!!!(might be ranked array logic bug)
ranked_df = ranked_df[ranked_df["unique_id"] != "497"]


In [7]:
ecoregions["unique_id"] = ecoregions["unique_id"].astype(str)

ecoregions = pd.merge(
    ecoregions, ranked_df, on="unique_id", how="outer", indicator=True
)

In [8]:
# replace nan with 0 in species_count column
ecoregions["species_count"] = ecoregions["species_count"].fillna(0)

# create a column for ranked_df called species_rank that ranks species by count, 1 being the highest
ecoregions["species_rank"] = ecoregions["species_count"].rank(
    ascending=False, method="first"
)

In [9]:
ecoregions.drop(["_merge", "_id"], axis=1, inplace=True)


In [12]:
# drop index from ecoregions
ecoregions.reset_index(drop=True, inplace=True)
